this makes sure that I can recover the predicted responses from saved parameters.

In [1]:
import os
from itertools import product
import numpy as np
import h5py

In [2]:
from tang_jcompneuro.io import load_split_dataset, load_split_dataset_idx
from tang_jcompneuro.model_fitting_postprocess import load_one_cnnpre_model
from tang_jcompneuro import dir_dictionary
from tang_jcompneuro.cnn_pretrained import blob_corresponding_info

In [3]:
blob_names = list(blob_corresponding_info['vgg19'].keys())

In [4]:
def check_one_layer(layer_name, seed):
    
    suffix=f'vgg19/legacy/{layer_name}'
    dataset = load_split_dataset('MkA_Shape', 'all', True, slice(None), last_val=False, suffix=suffix, seed=seed)
    dataset = dataset[4]
    print(dataset.shape)
    data_idx = load_split_dataset_idx('MkA_Shape', 'all', True, last_val=False, seed=seed)[2]
    assert data_idx.shape == (1900,)
    assert dataset.shape[0] == 1900 and dataset.ndim == 2
    
    # some random neurons.
    neurons_to_check = (0, 500, 231, 543)
    for neuron_idx in neurons_to_check:
        print(neuron_idx)
        model_this = load_one_cnnpre_model('MkA_Shape', 'all', suffix.replace('/','+'), neuron_idx, seed=seed)
        # ok. let's check
        # 
        y_test_hat = model_this['y_test_hat']
        assert y_test_hat.shape == (1900,1)
        assert model_this['model_coeff'].ndim == 1
        assert np.isscalar(model_this['model_bias'])
        y_test_hat_debug = np.exp(dataset@model_this['model_coeff'] + model_this['model_bias'])[:,np.newaxis]
        assert y_test_hat_debug.shape == y_test_hat.shape
#         print(abs(y_test_hat_debug-y_test_hat).max())
        assert abs(y_test_hat_debug-y_test_hat).max()<1e-10
        
    

In [5]:
def check_all():
    for layer_name, seed in product(['conv2_1', 'conv3_1', 'conv4_1'], range(2)):
        print(layer_name, seed)
        check_one_layer(layer_name, seed)

In [6]:
check_all()

conv2_1 0
(1900, 882)
0
500
231
543
conv2_1 1
(1900, 882)
0
500
231
543
conv3_1 0
(1900, 882)
0
500
231
543
conv3_1 1
(1900, 882)
0
500
231
543
conv4_1 0
(1900, 512)
0
500
231
543
conv4_1 1
(1900, 512)
0
500
231
543
